# Earthquake in Turkey

- Afad dataset


In [1]:
# Load up libraries

import pandas as pd
import folium
import datetime as dt
import sqlite3

print('Pandas: ', pd.__version__)
print('Folium: ', folium.__version__)

Pandas:  1.5.3
Folium:  0.14.0


## Data Source

https://tdvms.afad.gov.tr/event_spec_data

The dataset is being updated daily per events happened in Turkey and surounding areas.

## Forward Planning

The good part of this data source is we can grab down any events from the its historic database while specifying Lat/Long limits, Magnitude scopes, and time frame. I grabbed down the dataset from the beginning of year 2023 manually, and put/merge into a csv file named -
"data/afad.gov.tr-lat31-45-long21-48-yr2023.csv.csv". This is a very time consuming.

Thinking about -

* to build a local database (SQLite3?) to save what we have grabbed down so far,
* to run a webScraping task to grab down the daily updates from the website mentioned above,
* to merge the historic dataset and the realtime daily uodates into one database.
* 
* Finnally implement this into a .py file with a scheduled daily webscraper.
    

In [2]:
# Load u history data from csv file
histDF = pd.read_csv("afad.gov.tr-lat31-45-long21-48-yr2023.csv")
histDF.head()

,eventId,eventDate,longitude,latitude,depth,magnitudeType,magnitude,area
0,557556,2023-03-03T08:04:21,36.290,37.960,7.02,ML,3.9,Göksun (Kahramanmaraş)
1,557552,2023-03-03T07:42:12,36.896,37.318,15.61,ML,3.4,Nurdağı (Gaziantep)
2,557547,2023-03-03T07:40:53,38.757,38.327,8.40,ML,3.3,Kale (Malatya)
3,557525,2023-03-03T07:13:05,37.831,38.115,9.84,ML,3.0,Doğanşehir (Malatya)
4,557522,2023-03-03T07:11:13,37.184,37.472,9.29,ML,3.0,Dulkadiroğlu (Kahramanmaraş)


In [3]:
# Re-treatment of the dataset

# Renaming the columns per standardized data norm
histDF = histDF.rename(columns={'eventDate':'origintimeutc'})
histDF = histDF.rename(columns={'depth':'depthKm'})
histDF = histDF.rename(columns={'magnitudeType':'magType'})
histDF = histDF.rename(columns={'area':'region'})

# Adjust datetime from UTC (GMT) to Turkey timezone (GMT+3, or 'Europe/Istanbul')
histDF['origintimeutc'] = pd.to_datetime(histDF['origintimeutc'])
histDF['eventTime'] = histDF['origintimeutc'] + pd.DateOffset(hours=3)
histDF['eventTime'].dt.tz_localize(tz='Europe/Istanbul')

# Create new columns for date and time
histDF['date'] = pd.to_datetime(histDF['eventTime']).dt.date
histDF['time'] = pd.to_datetime(histDF['eventTime']).dt.time

# Add 2 more columns
histDF['measMethod'] = "Unknown"
histDF['attribute'] = "TDVMS"

# Subset the df and rename the columns
histDF = histDF[['origintimeutc', 'magnitude', 'magType', 'latitude', 'longitude', 'depthKm', 
                 'measMethod', 'region', 'attribute', 'eventId', 'eventTime', 'date', 'time']]
histDF.head()


,origintimeutc,magnitude,magType,latitude,longitude,depthKm,measMethod,region,attribute,eventId,eventTime,date,time
0,2023-03-03 08:04:21,3.9,ML,37.960,36.290,7.02,Unknown,Göksun (Kahramanmaraş),TDVMS,557556,2023-03-03 11:04:21,2023-03-03,11:04:21
1,2023-03-03 07:42:12,3.4,ML,37.318,36.896,15.61,Unknown,Nurdağı (Gaziantep),TDVMS,557552,2023-03-03 10:42:12,2023-03-03,10:42:12
2,2023-03-03 07:40:53,3.3,ML,38.327,38.757,8.40,Unknown,Kale (Malatya),TDVMS,557547,2023-03-03 10:40:53,2023-03-03,10:40:53
3,2023-03-03 07:13:05,3.0,ML,38.115,37.831,9.84,Unknown,Doğanşehir (Malatya),TDVMS,557525,2023-03-03 10:13:05,2023-03-03,10:13:05
4,2023-03-03 07:11:13,3.0,ML,37.472,37.184,9.29,Unknown,Dulkadiroğlu (Kahramanmaraş),TDVMS,557522,2023-03-03 10:11:13,2023-03-03,10:11:13


In [4]:
# print the earthquake with magnitude >=4
print(histDF[histDF['magnitude'].astype('float') >= 4])

           origintimeutc  magnitude magType  latitude  longitude  depthKm  \
14   2023-03-03 02:53:42        5.0      MW    37.847     36.602     7.06   
33   2023-03-01 16:38:00        4.3      ML    37.304     37.025     7.60   
41   2023-03-01 10:52:36        4.1      MW    41.304     43.858     4.31   
45   2023-03-01 07:20:22        4.4      MW    36.318     36.073    11.06   
46   2023-03-01 04:15:30        4.2      MW    37.833     37.856     7.00   
...                  ...        ...     ...       ...        ...      ...   
2862 2023-01-07 01:52:57        5.0      MW    39.374     26.249    14.02   
2869 2023-01-04 20:03:52        4.0      MW    38.835     23.630     6.88   
2870 2023-01-03 20:49:15        4.0      MW    37.210     22.509     6.50   
2874 2023-01-02 19:06:24        4.1      MW    35.339     26.222     8.69   
2876 2023-01-02 08:12:25        4.4      MW    35.062     26.211     6.51   

     measMethod                                             region attribut

In [5]:
# Define our functions
def addCircles(df, map):
    for x, y, z, loc, date, time in zip(df['latitude'], df['longitude'], df['magnitude'], df['region'], df['date'], df['time']):
        popUp = f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Magnitude: {z}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Location: {loc}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Date: {date}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Time: {time}</strong></span></p>"
        iframe = folium.IFrame(popUp, width=250, height=150)
        popup = folium.Popup(iframe, max_width=450)
        folium.CircleMarker(location=(x, y), radius=float(z) * 3, weight=2, opacity=1, popup=popup,
                            color="red", fill_color="red", fill_opacity=0.1).add_to(map)

In [6]:
# Setup initial map
magThreshold = 4
mapCtr = [39.16, 35.66]

eqMap = folium.Map(location=mapCtr, zoom_start=6, tiles="Stamen Terrain")

# Load up the polygon (in GeoJson format) around turkey
# Original URL was - https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json
folium.GeoJson('../data/PB2002_boundaries.json', name="geojson").add_to(eqMap)

# Add tiles
folium.TileLayer('Stamen Terrain').add_to(eqMap)
folium.TileLayer('Stamen Toner').add_to(eqMap)
folium.TileLayer('Stamen Water Color').add_to(eqMap)
folium.TileLayer('cartodbpositron').add_to(eqMap)
folium.TileLayer('cartodbdark_matter').add_to(eqMap)
folium.LayerControl().add_to(eqMap)

# Merge the dataframes: histDF + earthquakeDF
eqDF = histDF

# add Circles (sizing per magnitude)
eqDF = eqDF[eqDF['magnitude'].astype('float') >= magThreshold]
addCircles(eqDF, eqMap)

# Take a look at the map
eqMap


## The End